 # 🧪 Automated Experiments Runner
 סקריפט זה מריץ סדרת ניסויים להשוואת מודלים.
 הוא כולל:
 1. טעינת דאטה וסינון תת-קבוצה.
 2. הגדרת מודל דינמי (ResNet18/50).
 3. פונקציות Loss מתקדמות (BCE, Cosine).
 4. שמירת המודל המנצח מכל ניסוי.
 5. יצירת טבלת סיכום באקסל.

In [1]:
import os
import time
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import models
from tqdm import tqdm
from pathlib import Path

# --- הגדרות גלובליות ---
DATA_DIR = Path("DATA")
MODELS_DIR = Path("experiments_models")
MODELS_DIR.mkdir(exist_ok=True)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SUBSET_SIZE = 1500  # כמות התמונות לניסוי (כ-20% מהדאטה)
NUM_EPOCHS = 5      # מספר אפוקים לכל ניסוי (מספיק כדי לראות מגמה)

print(f"Running on: {DEVICE}")

Running on: cuda


# ## 1. Dataset Class
# טעינת התמונות, המסיכות והווקטורים.

In [2]:
class PotentialFlowDataset(Dataset):
    def __init__(self, root_dir, split='train', target_size=(512, 256)):
        self.root_dir = Path(root_dir)
        self.split = split
        self.target_size = target_size
        
        self.img_dir = self.root_dir / "images" / split
        self.mask_dir = self.root_dir / "masks" / split
        self.flow_dir = self.root_dir / "flow" / split
        
        self.images = sorted(list(self.img_dir.glob("*.jpg")))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        stem = img_path.stem
        
        # Load
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(str(self.mask_dir / f"{stem}.png"), cv2.IMREAD_GRAYSCALE)
        flow = np.load(str(self.flow_dir / f"{stem}.npy"))

        # Resize
        img = cv2.resize(img, self.target_size)
        mask = cv2.resize(mask, self.target_size, interpolation=cv2.INTER_NEAREST)
        flow = cv2.resize(flow, self.target_size)

        # To Tensor
        img_tensor = torch.from_numpy(img).float() / 255.0
        img_tensor = img_tensor.permute(2, 0, 1)
        
        mask_tensor = torch.from_numpy(mask).float() / 255.0
        mask_tensor = mask_tensor.unsqueeze(0)
        
        flow_tensor = torch.from_numpy(flow).float()
        flow_tensor = flow_tensor.permute(2, 0, 1)

        return img_tensor, mask_tensor, flow_tensor


#  2. Dynamic Model Architecture
 מודל שיודע להחליף את ה-Encoder שלו (ResNet18 או ResNet50) לפי בקשה.


In [3]:
class DynamicDualHeadModel(nn.Module):
    def __init__(self, backbone_name='resnet18'):
        super().__init__()
        if backbone_name == 'resnet18':
            base = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
            self.dims = [64, 64, 128, 256, 512]
        elif backbone_name == 'resnet50':
            base = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
            self.dims = [64, 256, 512, 1024, 2048]
        else:
            raise ValueError(f"Unknown backbone: {backbone_name}")
            
        layers = list(base.children())
        self.layer0 = nn.Sequential(*layers[:3])
        self.layer1 = nn.Sequential(*layers[3:5])
        self.layer2 = layers[5]
        self.layer3 = layers[6]
        self.layer4 = layers[7]
        
        # Decoder מותאם דינמית לגודל הערוצים
        self.up4 = self._up_block(self.dims[4], self.dims[3])
        self.up3 = self._up_block(self.dims[3]*2, self.dims[2])
        self.up2 = self._up_block(self.dims[2]*2, self.dims[1])
        self.up1 = self._up_block(self.dims[1]*2, 64)
        self.up0 = self._up_block(128, 32)
        
        # Heads
        self.potential_head = nn.Sequential(nn.Conv2d(32, 1, 1), nn.Sigmoid())
        self.flow_head = nn.Sequential(nn.Conv2d(32, 2, 1), nn.Tanh())

    def _up_block(self, in_c, out_c):
        return nn.Sequential(
            nn.ConvTranspose2d(in_c, out_c, 2, 2),
            nn.BatchNorm2d(out_c), nn.ReLU(inplace=True),
            nn.Conv2d(out_c, out_c, 3, padding=1),
            nn.BatchNorm2d(out_c), nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x0 = self.layer0(x)
        x1 = self.layer1(x0)
        x2 = self.layer2(x1)
        x3 = self.layer3(x2)
        x4 = self.layer4(x3)
        
        up4 = self.up4(x4)
        up3 = self.up3(torch.cat([up4, x3], 1))
        up2 = self.up2(torch.cat([up3, x2], 1))
        up1 = self.up1(torch.cat([up2, x1], 1))
        feat = self.up0(torch.cat([up1, x0], 1))
        
        return self.potential_head(feat), self.flow_head(feat)


#  3. Losses & Metrics
 הגדרת פונקציות הפסד (Cosine Similarity) ומדדי ביצועים (IoU, Precision, Recall, EPE).


In [4]:
class CosineFlowLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        self.cosine = nn.CosineSimilarity(dim=1)
        
    def forward(self, pred, target):
        # שילוב של דיוק בגודל (MSE) ודיוק בזווית (Cosine)
        return 0.7 * self.mse(pred, target) + 0.3 * (1 - self.cosine(pred, target).mean())

def calculate_metrics(pred_pot, target_pot, pred_flow, target_flow):
    # --- Segmentation Metrics (Potential) ---
    pred_bin = (pred_pot > 0.5).float()
    target_bin = (target_pot > 0.5).float()
    
    tp = (pred_bin * target_bin).sum().item()
    fp = (pred_bin * (1-target_bin)).sum().item()
    fn = ((1-pred_bin) * target_bin).sum().item()
    
    epsilon = 1e-6
    iou = tp / (tp + fp + fn + epsilon)
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)
    
    # --- Flow Metrics ---
    # EPE: End Point Error (מרחק אוקלידי ממוצע בין הוקטורים)
    diff = pred_flow - target_flow
    epe = torch.norm(diff, dim=1).mean().item()
    
    return iou, precision, recall, epe

def measure_fps(model):
    """מדידת מהירות חיזוי (פריימים לשנייה)"""
    model.eval()
    dummy = torch.randn(1, 3, 256, 512).to(DEVICE)
    # חימום
    for _ in range(10): 
        with torch.no_grad(): model(dummy)
        
    start = time.time()
    steps = 50
    for _ in range(steps): 
        with torch.no_grad(): model(dummy)
    return steps / (time.time() - start)


#  4. The Experiment Engine
 הפונקציה שמריצה ניסוי בודד, שומרת את המודל ומחזירה דוח.


In [5]:
def run_experiment(config):
    exp_name = config['name']
    print(f"\n🔥 Experiment: {exp_name}")
    print(f"   Config: Backbone={config['backbone']} | PotLoss={config['pot_loss']} | FlowLoss={config['flow_loss']} | Batch={config['batch']}")
    
    # 1. הכנת הדאטה (Subset לטובת מהירות)
    full_ds = PotentialFlowDataset(DATA_DIR, 'train')
    # בחירה אקראית של תמונות לניסוי
    indices = np.random.choice(len(full_ds), min(SUBSET_SIZE, len(full_ds)), replace=False)
    train_loader = DataLoader(Subset(full_ds, indices), batch_size=config['batch'], shuffle=True)
    
    # שימוש ב-Val המלא לדיוק
    val_loader = DataLoader(PotentialFlowDataset(DATA_DIR, 'val'), batch_size=config['batch'])
    
    # 2. הכנת המודל והאופטימייזר
    model = DynamicDualHeadModel(config['backbone']).to(DEVICE)
    optimizer = optim.AdamW(model.parameters(), lr=0.0001)
    
    # 3. בחירת פונקציות הפסד
    crit_pot = nn.BCELoss() if config['pot_loss'] == 'bce' else nn.MSELoss()
    crit_flow = CosineFlowLoss() if config['flow_loss'] == 'cosine' else nn.MSELoss()
    
    best_val_loss = float('inf')
    best_metrics = np.zeros(4) # iou, prec, rec, epe
    
    start_time = time.time()
    
    # 4. לולאת אימון
    for epoch in range(NUM_EPOCHS):
        # Train
        model.train()
        for imgs, masks, flows in train_loader:
            imgs, masks, flows = imgs.to(DEVICE), masks.to(DEVICE), flows.to(DEVICE)
            pot_out, flow_out = model(imgs)
            loss = crit_pot(pot_out, masks) + crit_flow(flow_out, flows)
            
            optimizer.zero_grad(); loss.backward(); optimizer.step()
            
        # Val & Metrics
        model.eval()
        val_loss = 0
        metrics_sum = np.zeros(4)
        
        with torch.no_grad():
            for imgs, masks, flows in val_loader:
                imgs, masks, flows = imgs.to(DEVICE), masks.to(DEVICE), flows.to(DEVICE)
                pot_out, flow_out = model(imgs)
                
                val_loss += (crit_pot(pot_out, masks) + crit_flow(flow_out, flows)).item()
                metrics_sum += calculate_metrics(pot_out, masks, flow_out, flows)
        
        val_loss /= len(val_loader)
        metrics_avg = metrics_sum / len(val_loader)
        
        print(f"   Ep {epoch+1}: Val Loss={val_loss:.4f} | IoU={metrics_avg[0]:.3f} | EPE={metrics_avg[3]:.3f}")
        
        # שמירת המודל הטוב ביותר בניסוי הנוכחי
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_metrics = metrics_avg
            # שמירה עם שם ייחודי לניסוי
            torch.save(model.state_dict(), MODELS_DIR / f"{exp_name}_best.pth")
            
    # מדידת FPS בסוף
    fps = measure_fps(model)
    total_time = (time.time() - start_time) / 60
    
    return {
        "Name": exp_name,
        "Backbone": config['backbone'],
        "Pot Loss": config['pot_loss'],
        "Flow Loss": config['flow_loss'],
        "Best Val Loss": round(best_val_loss, 4),
        "IoU": round(best_metrics[0], 3),
        "Precision": round(best_metrics[1], 3),
        "Recall": round(best_metrics[2], 3),
        "EPE (Flow Error)": round(best_metrics[3], 3),
        "FPS": round(fps, 1),
        "Time (min)": round(total_time, 1)
    }


#  5. Execution
 הגדרת רשימת הניסויים והרצה.


In [6]:
experiments_list = [
    # 1. Base: המצב הנוכחי (ResNet18 + MSE)
    {"name": "base_R18_MSE", "backbone": "resnet18", "pot_loss": "mse", "flow_loss": "mse", "batch": 8},
    
    # 2. Loss A: שיפור סגמנטציה עם BCE
    {"name": "base_R18_BCE", "backbone": "resnet18", "pot_loss": "bce", "flow_loss": "mse", "batch": 8},
    
    # 3. Loss B: שיפור מלא (BCE למפה + Cosine לווקטורים)
    {"name": "base_R18_FullLoss", "backbone": "resnet18", "pot_loss": "bce", "flow_loss": "cosine", "batch": 8},
    
    # 4. Heavy: מוח חזק יותר (ResNet50) עם הלוסים החדשים
    # שים לב: הורדנו Batch Size ל-4 כי המודל כבד יותר
    {"name": "heavy_R50_FullLoss", "backbone": "resnet50", "pot_loss": "bce", "flow_loss": "cosine", "batch": 4},
]

# הרצת הניסויים
all_results = []
print(f"--- Starting {len(experiments_list)} Experiments ---\n")

for exp_config in experiments_list:
    try:
        # הרצת הניסוי הבודד
        res = run_experiment(exp_config)
        all_results.append(res)
        
        # שמירת ביניים (כדי לא לאבד נתונים אם המחשב קורס)
        pd.DataFrame(all_results).to_csv("experiments_results.csv", index=False)
        print(f"✅ Finished {exp_config['name']}\n")
        
    except Exception as e:
        print(f"❌ Failed {exp_config['name']}: {e}\n")
        torch.cuda.empty_cache() # ניקוי זיכרון למקרה של נפילה

# סיכום סופי
print("\n🏆 ALL EXPERIMENTS COMPLETED!")
final_df = pd.DataFrame(all_results)
final_df.to_csv("experiments_results_final.csv", index=False)

# הצגת הטבלה בפורמט קריא (אם מריצים ב-Notebook/Interactive)
try:
    from IPython.display import display
    display(final_df)
except:
    print(final_df)
    

--- Starting 4 Experiments ---


🔥 Experiment: base_R18_MSE
   Config: Backbone=resnet18 | PotLoss=mse | FlowLoss=mse | Batch=8
   Ep 1: Val Loss=0.1432 | IoU=0.486 | EPE=0.246
   Ep 2: Val Loss=0.0943 | IoU=0.673 | EPE=0.187
   Ep 3: Val Loss=0.0682 | IoU=0.741 | EPE=0.156
   Ep 4: Val Loss=0.0594 | IoU=0.755 | EPE=0.142
   Ep 5: Val Loss=0.0518 | IoU=0.747 | EPE=0.132
✅ Finished base_R18_MSE


🔥 Experiment: base_R18_BCE
   Config: Backbone=resnet18 | PotLoss=bce | FlowLoss=mse | Batch=8
   Ep 1: Val Loss=0.3875 | IoU=0.601 | EPE=0.224
   Ep 2: Val Loss=0.3176 | IoU=0.752 | EPE=0.161
   Ep 3: Val Loss=0.2912 | IoU=0.740 | EPE=0.147
   Ep 4: Val Loss=0.2685 | IoU=0.795 | EPE=0.134
   Ep 5: Val Loss=0.2511 | IoU=0.797 | EPE=0.131
✅ Finished base_R18_BCE


🔥 Experiment: base_R18_FullLoss
   Config: Backbone=resnet18 | PotLoss=bce | FlowLoss=cosine | Batch=8
   Ep 1: Val Loss=0.6637 | IoU=0.622 | EPE=0.227
   Ep 2: Val Loss=0.5839 | IoU=0.630 | EPE=0.178
   Ep 3: Val Loss=0.5424 | IoU=0.6

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\guyy6/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 62.5MB/s]


   Ep 1: Val Loss=0.6164 | IoU=0.623 | EPE=0.192
   Ep 2: Val Loss=0.5248 | IoU=0.665 | EPE=0.161
   Ep 3: Val Loss=0.4738 | IoU=0.782 | EPE=0.142
   Ep 4: Val Loss=0.4494 | IoU=0.778 | EPE=0.126
   Ep 5: Val Loss=0.4307 | IoU=0.801 | EPE=0.120
✅ Finished heavy_R50_FullLoss


🏆 ALL EXPERIMENTS COMPLETED!


,Name,Backbone,Pot Loss,Flow Loss,Best Val Loss,IoU,Precision,Recall,EPE (Flow Error),FPS,Time (min)
0,base_R18_MSE,resnet18,mse,mse,0.0518,0.747,0.788,0.936,0.132,289.2,12.2
1,base_R18_BCE,resnet18,bce,mse,0.2511,0.797,0.863,0.910,0.131,308.5,12.4
2,base_R18_FullLoss,resnet18,bce,cosine,0.4892,0.737,0.759,0.961,0.140,188.8,11.6
3,heavy_R50_FullLoss,resnet50,bce,cosine,0.4307,0.801,0.877,0.901,0.120,90.6,13.8


# 🧪 Round 2: Advanced Loss & ResNet50 Focus
 בסיבוב הזה אנו מתמקדים במודל המנצח (ResNet50) ומנסים לשפר את ה-Flow שלו.
 בנוסף, אנו בודקים פונקציית Loss משולבת (Combo).


In [7]:
class ComboFlowLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        self.cosine = nn.CosineSimilarity(dim=1)
        
    def forward(self, pred, target):
        # MSE דואג שהגודל של החץ יהיה נכון
        loss_mse = self.mse(pred, target)
        # Cosine דואג שהזווית תהיה מדויקת
        loss_cos = 1 - self.cosine(pred, target).mean()
        
        # שילוב מאוזן: חצי-חצי
        return 0.5 * loss_mse + 0.5 * loss_cos

# 2. עדכון פונקציית ההרצה (כדי שתכיר את ה-Combo)
def run_experiment_v2(config):
    exp_name = config['name']
    print(f"\n🔥 Round 2 Experiment: {exp_name}")
    print(f"   Config: {config['backbone']} | FlowLoss: {config['flow_loss']}")
    
    # Data
    full_ds = PotentialFlowDataset(DATA_DIR, 'train')
    indices = np.random.choice(len(full_ds), min(SUBSET_SIZE, len(full_ds)), replace=False)
    train_loader = DataLoader(Subset(full_ds, indices), batch_size=config['batch'], shuffle=True)
    val_loader = DataLoader(PotentialFlowDataset(DATA_DIR, 'val'), batch_size=config['batch'])
    
    # Model
    model = DynamicDualHeadModel(config['backbone']).to(DEVICE)
    optimizer = optim.AdamW(model.parameters(), lr=0.0001)
    
    # Losses Selection
    crit_pot = nn.BCELoss() # תמיד BCE כי ראינו שזה מנצח
    
    if config['flow_loss'] == 'mse':
        crit_flow = nn.MSELoss()
    elif config['flow_loss'] == 'cosine':
        crit_flow = CosineFlowLoss()
    elif config['flow_loss'] == 'combo':
        crit_flow = ComboFlowLoss() # החדש!
    
    best_val_loss = float('inf')
    best_metrics = np.zeros(4)
    start_time = time.time()
    
    for epoch in range(NUM_EPOCHS):
        model.train()
        for imgs, masks, flows in train_loader:
            imgs, masks, flows = imgs.to(DEVICE), masks.to(DEVICE), flows.to(DEVICE)
            pot_out, flow_out = model(imgs)
            loss = crit_pot(pot_out, masks) + crit_flow(flow_out, flows)
            optimizer.zero_grad(); loss.backward(); optimizer.step()
            
        model.eval()
        val_loss = 0
        metrics_sum = np.zeros(4)
        with torch.no_grad():
            for imgs, masks, flows in val_loader:
                imgs, masks, flows = imgs.to(DEVICE), masks.to(DEVICE), flows.to(DEVICE)
                pot_out, flow_out = model(imgs)
                val_loss += (crit_pot(pot_out, masks) + crit_flow(flow_out, flows)).item()
                metrics_sum += calculate_metrics(pot_out, masks, flow_out, flows)
        
        val_loss /= len(val_loader)
        metrics_avg = metrics_sum / len(val_loader)
        
        print(f"   Ep {epoch+1}: Val Loss={val_loss:.4f} | EPE={metrics_avg[3]:.3f}")
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_metrics = metrics_avg
            torch.save(model.state_dict(), MODELS_DIR / f"{exp_name}_best.pth")
            
    fps = measure_fps(model)
    return {
        "Name": exp_name,
        "Backbone": config['backbone'],
        "Flow Loss": config['flow_loss'],
        "IoU": round(best_metrics[0], 3),
        "EPE (Flow Error)": round(best_metrics[3], 3),
        "FPS": round(fps, 1)
    }

# 3. רשימת הניסויים לסיבוב השני
round_2_experiments = [
    # בדיקה: האם MSE לבד עובד טוב יותר ב-R50?
    {"name": "R50_BCE_MSE", "backbone": "resnet50", "flow_loss": "mse", "batch": 4},
    
    # התקווה הגדולה: שילוב של השניים
    {"name": "R50_BCE_Combo", "backbone": "resnet50", "flow_loss": "combo", "batch": 4},
    
    # ניסיון להציל את המודל הקל עם הלוס החדש
    {"name": "R18_BCE_Combo", "backbone": "resnet18", "flow_loss": "combo", "batch": 8},
]

# 4. הרצה
results_r2 = []
print(f"--- Starting Round 2 ({len(round_2_experiments)} Experiments) ---\n")

for exp in round_2_experiments:
    try:
        res = run_experiment_v2(exp)
        results_r2.append(res)
        pd.DataFrame(results_r2).to_csv("results_round_2.csv", index=False)
        print(f"✅ Finished {exp['name']}\n")
    except Exception as e:
        print(f"❌ Failed {exp['name']}: {e}\n")
        torch.cuda.empty_cache()

print("\n🏆 ROUND 2 COMPLETED!")
df_r2 = pd.DataFrame(results_r2)

# איחוד עם התוצאות הקודמות (אם קיימות) כדי לראות הכל בטבלה אחת
if os.path.exists("experiments_results_final.csv"):
    df_old = pd.read_csv("experiments_results_final.csv")
    # התאמת עמודות (כי בחדש יש פחות פרטים)
    print("\n--- Combined Results (Top Models) ---")
    display(pd.concat([df_old, df_r2], ignore_index=True).sort_values("EPE (Flow Error)"))
else:
    display(df_r2)

--- Starting Round 2 (3 Experiments) ---


🔥 Round 2 Experiment: R50_BCE_MSE
   Config: resnet50 | FlowLoss: mse
   Ep 1: Val Loss=0.4893 | EPE=0.212
   Ep 2: Val Loss=0.3737 | EPE=0.163
   Ep 3: Val Loss=0.3122 | EPE=0.133
   Ep 4: Val Loss=0.2715 | EPE=0.128
   Ep 5: Val Loss=0.2506 | EPE=0.117
✅ Finished R50_BCE_MSE


🔥 Round 2 Experiment: R50_BCE_Combo
   Config: resnet50 | FlowLoss: combo
   Ep 1: Val Loss=0.8181 | EPE=0.261
   Ep 2: Val Loss=0.7091 | EPE=0.174
   Ep 3: Val Loss=0.6398 | EPE=0.153
   Ep 4: Val Loss=0.6038 | EPE=0.143
   Ep 5: Val Loss=0.5807 | EPE=0.137
✅ Finished R50_BCE_Combo


🔥 Round 2 Experiment: R18_BCE_Combo
   Config: resnet18 | FlowLoss: combo
   Ep 1: Val Loss=0.7859 | EPE=0.307
   Ep 2: Val Loss=0.6904 | EPE=0.211
   Ep 3: Val Loss=0.6479 | EPE=0.174
   Ep 4: Val Loss=0.6251 | EPE=0.159
   Ep 5: Val Loss=0.6051 | EPE=0.148
✅ Finished R18_BCE_Combo


🏆 ROUND 2 COMPLETED!

--- Combined Results (Top Models) ---


,Name,Backbone,Pot Loss,Flow Loss,Best Val Loss,IoU,Precision,Recall,EPE (Flow Error),FPS,Time (min)
4,R50_BCE_MSE,resnet50,NaN,mse,NaN,0.795,NaN,NaN,0.117,88.9,NaN
3,heavy_R50_FullLoss,resnet50,bce,cosine,0.4307,0.801,0.877,0.901,0.120,90.6,13.8
1,base_R18_BCE,resnet18,bce,mse,0.2511,0.797,0.863,0.910,0.131,308.5,12.4
0,base_R18_MSE,resnet18,mse,mse,0.0518,0.747,0.788,0.936,0.132,289.2,12.2
5,R50_BCE_Combo,resnet50,NaN,combo,NaN,0.779,NaN,NaN,0.137,95.9,NaN
2,base_R18_FullLoss,resnet18,bce,cosine,0.4892,0.737,0.759,0.961,0.140,188.8,11.6
6,R18_BCE_Combo,resnet18,NaN,combo,NaN,0.780,NaN,NaN,0.148,197.3,NaN
